In [1]:
import numpy as np
from scipy.stats import norm

class OptionPricing:
    def __init__(self, spot_price, strike_price, maturity, volatility, interest_rate):
        self.spot_price = spot_price
        self.strike_price = strike_price
        self.maturity = maturity
        self.volatility = volatility
        self.interest_rate = interest_rate

    def generate_paths(self, num_paths, num_steps):
        dt = self.maturity / num_steps
        sqrt_dt = np.sqrt(dt)

        paths = np.zeros((num_paths, num_steps + 1))
        paths[:, 0] = self.spot_price

        for t in range(1, num_steps + 1):
            dW = np.random.normal(0, sqrt_dt, num_paths)
            paths[:, t] = paths[:, t - 1] * np.exp((self.interest_rate - 0.5 * self.volatility ** 2) * dt
                                                    + self.volatility * sqrt_dt * dW)

        return paths

    def european_call_option_price(self, num_paths=10000, num_steps=252):
        paths = self.generate_paths(num_paths, num_steps)
        option_payoffs = np.maximum(paths[:, -1] - self.strike_price, 0)
        discounted_payoffs = np.exp(-self.interest_rate * self.maturity) * option_payoffs

        option_price = np.mean(discounted_payoffs)
        return option_price

    def american_call_option_price(self, num_paths=10000, num_steps=252):
        paths = self.generate_paths(num_paths, num_steps)
        option_payoffs = np.maximum(paths - self.strike_price, 0)

        for t in range(num_steps - 1, 0, -1):
            intrinsic_value = np.maximum(paths[:, t] - self.strike_price, 0)
            continuation_value = np.exp(-self.interest_rate * (self.maturity - t / num_steps)) * \
                                 np.mean(option_payoffs[:, t + 1])

            option_payoffs[:, t] = np.maximum(intrinsic_value, continuation_value)

        discounted_payoffs = np.exp(-self.interest_rate * self.maturity) * option_payoffs[:, 1]

        option_price = np.mean(discounted_payoffs)
        return option_price

# Example usage:
spot_price = 100
strike_price = 100
maturity = 1
volatility = 0.2
interest_rate = 0.05

option_pricing = OptionPricing(spot_price, strike_price, maturity, volatility, interest_rate)

european_call_price = option_pricing.european_call_option_price()
american_call_price = option_pricing.american_call_option_price()

print("European Call Option Price:", european_call_price)
print("American Call Option Price:", american_call_price)


European Call Option Price: 2.881535847806567
American Call Option Price: 0.33633403283830593
